<a href="https://www.kaggle.com/upamanyumukherjee/boston-dataset?scriptVersionId=88622001" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <font color=darkgreen> Building a Deep Neural Network for the Boston Housing Price dataset </font>

This dataset is relatively smaller than the previous datasets with only 506 data points (303 for training and 102 for testing,101 for validation).
Each data point has a set of 13 features.

**Using all types of Sequential Neural Networks to see which model works best with this model dataset.**

In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

In [ ]:
print(train_data.shape)
print(test_data.shape)

# **Data Preprocess**

In [ ]:
# Normalize the data by subtracting the mean from each data point and
# dividing by the standard deviation of the data

mean = train_data.mean(axis=0) # since we want the mean for each feature column
print('Mean =', mean)
train_data -= mean

std_dev = train_data.std(axis=0)
print('Std Dev = ', std_dev)
train_data /= std_dev

# Likewise prepare the test data (pre-processing)
test_data -= mean
test_data /= std_dev

# **Holdout validation**
# Creating a validation dataset from the train dataset thus forming the Holdout validation as we already have a test to evaluate on

In [ ]:
from sklearn.model_selection import train_test_split
#split dataset into train and test data
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.25, random_state=100)

In [ ]:
val_data.shape

In [ ]:
train_data.shape

In [ ]:
train_labels.shape

In [ ]:
val_labels.shape

In [ ]:
train_labels.shape

In [ ]:
# View the training labels
# Prices are in 10,000s
print(train_labels[1:5])

**We havetaken in Boston housing price 4 Dense [32, 32, 64, 1 ] relu in the first 3 layers and no activation in the final layer rmsprop lr=0.0001**

In [ ]:
model = keras.Sequential([ 
        tf.keras.layers.Dense(32, activation='relu',input_shape=(404,13)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
optimizer=tf.keras.optimizers.RMSprop(lr=0.0001)
model.compile(optimizer= optimizer, loss='mse', metrics=['mae']) # loss='mse' and metrics='mae'

In [ ]:
model.summary()

**To choose the appropriate epoch no I choose Early stopping to avoid overfitting**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=1,restore_best_weights=True)

In [ ]:
history=model.fit(train_data, train_labels, epochs=200, batch_size=16,validation_data=(val_data,val_labels),callbacks=[early_stop])

In [ ]:
[mse, mae] = model.evaluate(test_data, test_labels) 

In [ ]:
ypred = model.predict(test_data)
print('Actual Price = {} and Predicted Price = {}'.format(test_labels[1], ypred[1]))

In [ ]:
# Let us plot the loss and accuracy curves
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
mae = history_dict['mae']
val_mae = history_dict['val_mae']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, mae, 'b', label='Training MAE')
plt.plot(epochs, val_mae, 'r', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

**There is no significant overfitting or underfitting in this dataset based on the plots**

### Using K-fold validation

In [ ]:
# We create a fucntion to make it easy for multiple calls
def build_model():   
    model = keras.Sequential([ 
        tf.keras.layers.Dense(32, activation='relu',input_shape=(404,13)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    optimizer=tf.keras.optimizers.RMSprop(lr=0.0001)
    model.compile(optimizer= optimizer, loss='mse', metrics=['mae']) # observe the loss and metrics
    return model

In [ ]:
# Let us visit K-fold validation
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #%d' % i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples] #taking data from a range of kth to kth +1 samples
    val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate( 
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)#taking data from a range of kth to kth +1 samples
    partial_train_labels = np.concatenate(
        [train_labels[:i * num_val_samples],
         train_labels[(i + 1) * num_val_samples:]],
        axis=0)
    
model = build_model() 
model.fit(partial_train_data, partial_train_labels, 
          epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_labels, verbose=0) 
all_scores.append(val_mae)

In [ ]:
print(all_scores)
print(np.mean(all_scores))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=1,restore_best_weights=True)

In [ ]:
model = build_model() 
history=model.fit(train_data, train_labels, epochs=280, batch_size=16,validation_data=(val_data,val_labels),callbacks=[early_stop])

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_labels)
print(test_mse_score, test_mae_score)

In [ ]:
history

In [ ]:
ypred = model.predict(test_data)
print('Actual Price = {} and Predicted Price = {}'.format(test_labels[1], ypred[1]))

In [ ]:
history_dict

In [ ]:
# Let us plot the loss and accuracy curves
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
mae = history_dict['mae']
val_mae = history_dict['val_mae']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, mae, 'b', label='Training Accuracy')
plt.plot(epochs, val_mae, 'r', label='Validation Accuracy')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

# **DNN-BatchNormalization**[](http://)

In [ ]:
model = keras.Sequential([ 
        tf.keras.layers.Dense(32, activation='relu',input_shape=(13,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
optimizer=tf.keras.optimizers.SGD(lr=0.0001)
model.compile(optimizer= optimizer, loss='mse', metrics=['mae']) # loss='mse' and metrics='mae'

In [ ]:
model.summary()

In [ ]:
history=model.fit(train_data, train_labels, epochs=180, batch_size=16,validation_data=(val_data,val_labels),callbacks=[early_stop])

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_labels)
print(test_mse_score, test_mae_score)

In [ ]:
ypred = model.predict(test_data)
print('Actual Price = {} and Predicted Price = {}'.format(test_labels[3], ypred[3]))

In [ ]:
# Let us plot the loss and accuracy curves
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
mae = history_dict['mae']
val_mae = history_dict['val_mae']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, mae, 'b', label='Training Accuracy')
plt.plot(epochs, val_mae, 'r', label='Validation Accuracy')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

# **GRU**

In [ ]:
train_data_GRU = np.reshape(train_data, (train_data.shape[0],train_data.shape[1],1))
model =  keras.Sequential([ 
        tf.keras.layers.GRU(16, return_sequences = True, activation='relu',input_shape=(train_data_GRU.shape[1],1)),
        tf.keras.layers.GRU(16, return_sequences = True, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])

In [ ]:
model.summary()

In [ ]:
optimizer=tf.keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae']) # observe the loss and metrics

In [ ]:
val_data_GRU = np.reshape(val_data, (val_data.shape[0],val_data.shape[1],1))

In [ ]:
history=model.fit(train_data_GRU, train_labels, epochs=130, batch_size=16,validation_data=(val_data_GRU, val_labels),callbacks=[early_stop])

In [ ]:
test_data_GRU = np.reshape(test_data, (test_data.shape[0],test_data.shape[1],1))

In [ ]:
[mse, mae] = model.evaluate(test_data_GRU, test_labels) 

In [ ]:
ypred = model.predict(test_data_GRU)
print('Actual Price = {} and Predicted Price = {}'.format(test_labels[1], ypred[1]))

In [ ]:
# Let us plot the loss and accuracy curves
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
mae = history_dict['mae']
val_mae = history_dict['val_mae']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, mae, 'b', label='Training MAE')
plt.plot(epochs, val_mae, 'r', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **CONV1D**

In [ ]:
train_data_Conv1d = np.reshape(train_data, (train_data.shape[0],train_data.shape[1],1))#This is to reshape the x input data. We'll create one-dimensional vectors from each row of x input data.
model =  keras.Sequential([ 
        tf.keras.layers.Conv1D(64, 1, activation='relu',input_shape=(train_data_Conv1d.shape[1],1)),
        tf.keras.layers.Conv1D(64, 1, activation='relu'),
        tf.keras.layers.MaxPooling1D(),
        tf.keras.layers.Dropout(.5),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])

In [ ]:
model.summary()

In [ ]:
optimizer=tf.keras.optimizers.SGD(lr=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae']) # observe the loss and metrics

In [ ]:
val_data_Con1D = np.reshape(val_data, (val_data.shape[0],val_data.shape[1],1))

In [ ]:
history=model.fit(train_data_Conv1d, train_labels, epochs=130, batch_size=16,validation_data=(val_data_Con1D, val_labels),callbacks=[early_stop])

In [ ]:
test_data_Conv1D = np.reshape(test_data, (test_data.shape[0],test_data.shape[1],1))

In [ ]:
[mse, mae] = model.evaluate(test_data_Conv1D, test_labels) 

In [ ]:
ypred = model.predict(test_data_Conv1D)
print('Actual Price = {} and Predicted Price = {}'.format(test_labels[1], ypred[1]))

In [ ]:
# Let us plot the loss and accuracy curves
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
mae = history_dict['mae']
val_mae = history_dict['val_mae']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, mae, 'b', label='Training MAE')
plt.plot(epochs, val_mae, 'r', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# **LSTM-holdout**

In [ ]:
train_data = np.reshape(train_data, (train_data.shape[0],train_data.shape[1],1))
model =  keras.Sequential([ 
        tf.keras.layers.LSTM(16, return_sequences = True, activation='relu',input_shape=(train_data.shape[1],1)),
        tf.keras.layers.LSTM(16, return_sequences = True, activation='relu'),
        tf.keras.layers.LSTM(16, return_sequences = True, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1)
    ])

In [ ]:
model.summary()

In [ ]:
optimizer=tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae']) # observe the loss and metrics

In [ ]:
val_data = np.reshape(val_data, (val_data.shape[0],val_data.shape[1],1))

In [ ]:
history=model.fit(train_data, train_labels, epochs=130, batch_size=16,validation_data=(val_data, val_labels),callbacks=[early_stop])

In [ ]:
test_data = np.reshape(test_data, (test_data.shape[0],test_data.shape[1],1))

In [ ]:
[mse, mae] = model.evaluate(test_data, test_labels) 

In [ ]:
ypred = model.predict(test_data)
print('Actual Price = {} and Predicted Price = {}'.format(test_labels[2], ypred[2]))

In [ ]:
# Let us plot the loss and accuracy curves
history_dict = history.history
loss_value = history_dict['loss']
val_loss_value = history_dict['val_loss']
mae = history_dict['mae']
val_mae = history_dict['val_mae']
epochs = range(1, len(loss_value) + 1)
plt.plot(epochs, loss_value, 'b', label='Training Loss')
plt.plot(epochs, val_loss_value, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure()

plt.plot(epochs, mae, 'b', label='Training MAE')
plt.plot(epochs, val_mae, 'r', label='Validation MAE')
plt.title('Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

$$
\begin{bmatrix}
    \hline
    \mathtt{Model} & \mathtt{MAE-Train} & \mathtt{MAE-Test} \\
    \hline 
    DNN-Holdout Validation & 2.1905 & 2.8366 \\  
    DNN-K-fold Validation & 2.6342 & 3.1717 \\ 
    DNN with BatchNormalization-Holdout Validation & 3.3871 & 2.701 \\
    GRU-Holdout Validation &  5.74 & 7.06 \\
    Conv1D-Holdout Validation & 6.309 & 6.67 \\
    LSTM-Holdout Validation &  7.24 & 8.99 \\ 
    \hline
\end{bmatrix}
$$

Hence we conclude that atleast in this case simple DNN is enough no need to go to GRU or LSTM specially due to some signs of overfitting it can be due to the data in this dataset too low for these models to work well. On the other hand DNN- KFold Validation or DNN with Batch Normalization can be also used as it gives results closer to real prices without overfitting. Lasltly although Conv1D doesn't overfit or underfit but result score is not enough so using simple case of DNN with Holdout Validation should work here.